In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy

In [3]:
num_epochs = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data_transforms = transforms.Compose(
     # transform image to tensor (multiple channels/dimensions)
     [transforms.Resize((224,224)),transforms.ToTensor(),
      # normalize image range from 0 and 255 to between 0 and 1
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

In [4]:
trainset = torchvision.datasets.CIFAR100('./data_set', train = True, transform = data_transforms, download=True)
testset = torchvision.datasets.CIFAR100('./data_set', train=False, transform = data_transforms, download=True)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data_set/cifar-100-python.tar.gz to ./data_set
Files already downloaded and verified


In [5]:
train_data = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=2) #If your OS is Windows, set num_workers be 0.
model = models.vgg16(pretrained = True)

num_ftrs=model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 100) # num_cls is the number of classes. Here it is the last layer, we want it to predict num_cls classes.

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [6]:
for param in model.parameters(): # freeze
    param.requires_grad = False
for param in model.classifier[6].parameters(): # train the last linear layer.
    param.requires_grad = True

model = model.to(device)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [8]:
for epoch in range(num_epochs):
    # loop over mini-batch
    print('Epoch: {0}'.format(epoch))

    for data in (train_data):
        inputs = data[0].to(device)
        labels = data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()



Epoch: 0
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9


In [9]:
best_model_wts = copy.deepcopy(model.state_dict())
torch.save(best_model_wts , 'best_model_weight.pth')

In [10]:
model.load_state_dict(torch.load('best_model_weight.pth'))
test_data = torch.utils.data.DataLoader(testset, batch_size=16, shuffle=True, num_workers=2) #If your OS is Windows, set num_workers be 0.


In [11]:
correct = 0
total = 0
for data in test_data:
  inputs, labels = data[0].to(device), data[1].to(device)
  outputs = model(inputs)
  _, predicted = torch.max(outputs.data, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()

print('Correct Predictions: {0}'.format(correct))
print('Total Predictions: {0}'.format(total))
print('Accuracy of the model: %d %%' % (100 * correct / total))

Correct Predictions: 4993
Total Predictions: 10000
Accuracy of the model: 49 %
